In [2]:
import pandas as pd
import numpy as np
import yfinance as yf

In [51]:
fund = pd.read_parquet('../data/raw/equity_fundamental_data.parquet')

In [ ]:
# spark_process.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, sum as spark_sum

# Start Spark session
spark = SparkSession.builder \
    .appName("Sales Data Processing") \
    .getOrCreate()

# Read the raw scraped data
df = spark.read.parquet("raw_sales_data.parquet")

# Add year and month columns
df = df.withColumn("year", year(col("date")))
df = df.withColumn("month", month(col("date")))

# Aggregate sales by category, year, month
agg_df = df.groupBy("category", "year", "month") \
           .agg(spark_sum("sales").alias("total_sales"))

# Save to a new Parquet file
agg_df.write.mode("overwrite").parquet("aggregated_sales.parquet")

spark.stop()
print("✅ Aggregated data saved.")
